#### Course Note Data Processing

In [ ]:
from course_notes_extraction import PDFToQAConverter

converter = PDFToQAConverter(api_key="your-gemini-api-key")

converter.process_directory(
    pdf_directory="data/design_and_analysis_of_algorithms/course_notes/pdfs",  # input raw pdfs directory
    output_json="data/design_and_analysis_of_algorithms/course_notes/json/design_combined.json"          # output file name
)


Processing PDFs: 100%|██████████| 3/3 [00:34<00:00, 11.63s/it]

Successfully saved 46 Q&A pairs to data/design_and_analysis_of_algorithms/course_notes/json/design_combined.json


{'questions': [{'question': 'What is the focus of Lecture 18?',
   'answer': 'Fixed-Parameter Algorithms, covering Vertex Cover, Fixed-Parameter Tractability, Kernelization, and Connection to Approximation.'},
  {'question': 'What are Fixed Parameter Algorithms and their desired features?',
   'answer': 'Fixed Parameter Algorithms are an alternative way to deal with NP-hard problems instead of approximation algorithms, aiming to solve NP-hard problems, run in polynomial time, and get exact solutions. In general, unless P = NP, an algorithm can have two of these three features, but not all three.'},
  {'question': 'Explain the idea behind using a specific parameter in Fixed Parameter Algorithms.',
   'answer': 'The idea is to aim for an exact algorithm but isolate exponential terms to a specific parameter. When the value of this parameter is small, the algorithm gets fast instances. Hopefully, this parameter will be small in practice.'},
  {'question': "Define 'Parameter' and 'Parameter

#### Exam QA Data Processing

In [ ]:
from exam_qa_extraction import PDFQuestionExtractor, PDFProcessor
from utils.data_util import combine_json_files

extractor = PDFQuestionExtractor(api_key="your-gemini-api-key")
processor = PDFProcessor(extractor)
processor.process_directory("data/design_and_analysis_of_algorithms/exams/pdfs", "data/design_and_analysis_of_algorithms/exams/json")
combine_json_files(input_dir="data/design_and_analysis_of_algorithms/exams/json", output_file="data/design_and_analysis_of_algorithms/exams/design_combined.json")

Processing PDFs:  50%|█████     | 1/2 [00:26<00:26, 26.14s/it]

Saved results to data/design_and_analysis_of_algorithms/exams/json\44146f3104fa80a52514265d070ebc40_MIT6_046JS15_quiz2sols_processed.json


Processing PDFs: 100%|██████████| 2/2 [00:55<00:00, 27.78s/it]


Saved results to data/design_and_analysis_of_algorithms/exams/json\5b6cd99674ea0c866c942b72aa9e7289_MIT6_046JS15_quiz1sols_processed.json


Processing files in data/design_and_analysis_of_algorithms/exams/json: 100%|██████████| 2/2 [00:00<00:00, 26.32it/s]

Combined 40 questions into data/design_and_analysis_of_algorithms/exams/design_combined.json


#### Create prompt datasets (n-prompt)

In [4]:
from create_prompt_dataset import create_chatgpt_input_dataset
create_chatgpt_input_dataset(input_file="data/design_and_analysis_of_algorithms/exams/design_combined.json", output_file="data/design_and_analysis_of_algorithms/0_shot.jsonl", shots=0)

Batch .jsonl file created: data/design_and_analysis_of_algorithms/0_shot.jsonl with 40 entries using 0-shot learning


#### Generate LLM Responses

In [1]:
import openai as client
import time
from utils.open_ai_batch_processing import upload_and_create_batch, save_batch_response

batch = upload_and_create_batch(jsonl_file_path="data/design_and_analysis_of_algorithms/0_shot.jsonl")
print(batch)
save_batch_response(batch, output_file="data/design_and_analysis_of_algorithms/llm_responses/0_shot.jsonl")

File uploaded successfully. File ID: file-6TDYY7psuRq9zeZQXJitdR
Batch created successfully. Batch ID: batch_67e5c3fc7d148190802cd73992d74463
Batch Status: validating
Request Counts: BatchRequestCounts(completed=0, failed=0, total=0)
{'file_id': 'file-6TDYY7psuRq9zeZQXJitdR', 'batch_id': 'batch_67e5c3fc7d148190802cd73992d74463', 'batch_status': 'validating'}
pending...
pending...
pending...
pending...
pending...
pending...
pending...
Batch output downloaded to data/design_and_analysis_of_algorithms/llm_responses/0_shot.jsonl


#### RAG with Course Notes

In [3]:
from rag import RAGPipeline
rag_pipeline = RAGPipeline()

# Load and process documents
rag_pipeline.load_and_process_documents(document_json_path="data/design_and_analysis_of_algorithms/exams/design_combined.json")

# Process input file and generate responses
rag_pipeline.process_jsonl_file(
    input_file="data/design_and_analysis_of_algorithms/0_shot.jsonl",
    output_file="data/design_and_analysis_of_algorithms/llm_responses/rag.jsonl"
)

c:\Users\Chris Joel\Desktop\cs-research-project\reformatted\rag.py:72: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  return self.qa_chain.run(question)


#### Evaluation (Rouge-1, Rouge-L, BLEU, BertScore, F1, Jaccard)

In [4]:
from evaluation import NLPEvaluator

evaluator = NLPEvaluator()
evaluator.evaluate(
    llm_response_file="data/design_and_analysis_of_algorithms/llm_responses/rag.jsonl",
    ground_truth_file="data/design_and_analysis_of_algorithms/exams/design_combined.json"
)

c:\Users\Chris Joel\Desktop\cs-research-project\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.de

Evaluation Results:
| Question ID       | ROUGE-1 | ROUGE-L | BLEU | BERTScore | F1 | Jaccard |
|-------------------|---------|---------|------|-----------|----|---------|
| 441462sols000     |    6.90% |    6.90% |   0.34% |     84.51% | 10.71% |    5.66% |
| 441462sols001     |   17.72% |   17.72% |   1.06% |     87.33% | 22.73% |   12.82% |
| 441462sols002     |   14.55% |   14.55% |   0.51% |     83.16% | 12.77% |    6.82% |
| 441462sols003     |   16.67% |   12.50% |   0.62% |     85.25% | 23.33% |   13.21% |
| 441462sols004     |   14.29% |   10.71% |   0.33% |     84.03% | 16.00% |    8.70% |
| 441462sols005     |   47.89% |   36.62% |  12.45% |     89.49% | 42.86% |   27.27% |
| 441462sols006     |   16.06% |   14.60% |   1.10% |     85.09% | 15.56% |    8.43% |
| 441462sols007     |   64.29% |   64.29% |  32.92% |     93.68% | 85.00% |   73.91% |
| 441462sols008     |   29.55% |   25.00% |  10.59% |     89.34% | 30.00% |   17.65% |
| 441462sols009     |   11.76% |    7.06% |  

{'441462sols000': EvaluationMetrics(rouge1=6.896551724137931, rougeL=6.896551724137931, bleu=0.34170206403382564, bertscore=84.51013565063477, f1=10.714285714285714, jaccard=5.660377358490567),
 '441462sols001': EvaluationMetrics(rouge1=17.72151898734177, rougeL=17.72151898734177, bleu=1.0606948946692587, bertscore=87.32571601867676, f1=22.727272727272727, jaccard=12.82051282051282),
 '441462sols002': EvaluationMetrics(rouge1=14.545454545454545, rougeL=14.545454545454545, bleu=0.5145174324020216, bertscore=83.16062688827515, f1=12.765957446808512, jaccard=6.8181818181818175),
 '441462sols003': EvaluationMetrics(rouge1=16.666666666666664, rougeL=12.5, bleu=0.6235344421390427, bertscore=85.24689078330994, f1=23.33333333333333, jaccard=13.20754716981132),
 '441462sols004': EvaluationMetrics(rouge1=14.285714285714285, rougeL=10.714285714285715, bleu=0.3333421816571239, bertscore=84.02745127677917, f1=16.0, jaccard=8.695652173913043),
 '441462sols005': EvaluationMetrics(rouge1=47.8873239436

#### LLM Assisted Evaluation

In [2]:
from llm_assisted_evaluation import evaluate

evaluate(
    llm_response_file="data/design_and_analysis_of_algorithms/llm_responses/rag.jsonl",
    ground_truth_file="data/design_and_analysis_of_algorithms/exams/design_combined.json",
)

LLM-Assisted Evaluation Results:
| Question ID       | Score | Comments |
|-------------------|-------|----------|
| 441462sols000 | 1 | The LLM correctly identifies the statement as false, which is in line with the ground truth. However, the explanation provided by the LLM contains inaccuracies. It states that O(V^3) is faster than O(Vlog2 7), which is incorrect, and it doesn't address the reason related to Strassen’s algorithm or the concept of defining negation. Therefore, the response is only partially correct. |
| 441462sols001 | 0 | The LLM response incorrectly concludes that Floyd-Warshall is asymptotically faster. The ground truth correctly states that Johnson's algorithm is asymptotically faster since its complexity O(VE + V² log V) becomes O(V³) when E = O(V^1.5), matching with Floyd-Warshall's O(V³) but is lower bounded, allowing Johnson to potentially be faster or equal depending on implementation factors. The faulty nesting of complexity reasoning leads to a 'wrong' score.

#### Fine-tuning
https://platform.openai.com/finetune/